In [315]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re
from sklearn.feature_extraction.text import CountVectorizer

In [316]:
all_verbs = pd.read_csv('./all_verbs_LR.csv').reset_index(drop = True)
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e
...,...,...,...,...,...,...,...,...,...,...,...,...
34475,34475,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
34476,34476,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
34477,34477,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ
34478,34478,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ


In [317]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e
...,...,...,...,...,...,...,...,...,...,...,...,...
34475,34475,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
34476,34476,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
34477,34477,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ
34478,34478,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ


In [318]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e
...,...,...,...,...,...,...,...,...,...,...,...,...
34475,34475,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
34476,34476,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
34477,34477,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ
34478,34478,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ


In [319]:
all_verbs[~all_verbs['Generated Verb-SR'].str.contains('ˈ')].head(30)

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion


In [320]:
all_verbs['Stress Number'] = all_verbs['Generated Verb-SR'].apply(lambda word: word.rindex('ˈ') / len(word)) #before : goes input variable
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34475,34475,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455
34476,34476,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455
34477,34477,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000
34478,34478,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000


In [321]:
from scipy.sparse import hstack
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
X = count_vect.fit_transform(all_verbs['Generated Verb-SR'])


In [322]:
hstack([X, all_verbs['Stress Number'].values.reshape(-1,1)]) 

<34480x1670 sparse matrix of type '<class 'numpy.float64'>'
	with 1241255 stored elements in COOrdinate format>

In [323]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:


        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'Stress Number']] 
        y = all_verbs[feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Stress Number'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Stress Number'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        #word_predict = model.predict(count_vect.transform(np.array(['ʌbɛmdinijɛst*ɛ*vijɛ'])))
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [324]:
log_with_stress_feature = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [325]:
log_with_stress_feature

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9995605940768082,
  'confusion matrix': array([[7959,    3],
         [   2, 3415]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9914755250900782,
  'confusion matrix': array([[ 185,    0,   43],
         [   3, 7930,    0],
         [  50,    1, 3167]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.973811406977766,
  'confusion matrix': array([[3094,    7,    0,    0],
         [  11, 3365,  114,    0],
         [   0,  166, 3010,    0],
         [   0,    0,    0, 1612]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999121188153617,
  'confusion matrix': array([[3579,    0],
         [   1, 7799]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993848317075315,
  'confusion matrix': array([[5715,    2],
         [   5, 5657]])},
 'Person': {'model': LogisticRegression(max_iter=100000

In [326]:
all_verbs['First Character'] = all_verbs['Generated Verb-SR'].apply(lambda word: 0 if word[0]in ['a', 'o', 'u', 'i','ø', 'ɛ', 'ʌ', 'e'] else 1) #before : goes input variable

In [327]:
all_verbs[all_verbs['First Character'] == 1]

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character
200,200,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,me-*ne*-andʌd͡ʒ-i-em(ɛ),menaˈndʌd͡ʒijemɛ,a,0.250000,1
201,201,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,me-*ne*-andʌd͡ʒ-i-i,menaˈndʌd͡ʒii,a,0.307692,1
202,202,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,me-*ne*-andʌd͡ʒ-i-e,menaˈndʌd͡ʒije,a,0.285714,1
203,203,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,me-*ne*-andʌd͡ʒ-i-iɛ,menaˈndʌd͡ʒiijɛ,a,0.266667,1
204,204,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,me-*ne*-andʌd͡ʒ-i-om(ɛ),menaˈndʌd͡ʒijomɛ,a,0.250000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32169,32169,past,subjunctive,perfect,non-causative,affirmative,3sm,passive,intrans,be-van-i-ast-*abe*,bevɛnijastaˈbe,a,0.785714,1
32170,32170,past,subjunctive,perfect,non-causative,affirmative,3sf,passive,intrans,be-van-i-ast-*abiɛ*,bevɛnijɛstɛˈbijɛ,ɛ,0.687500,1
32171,32171,past,subjunctive,perfect,non-causative,affirmative,1p,passive,intrans,be-van-i-ast-*abimon*,bevɛnijɛstɛˈbimon,ɛ,0.647059,1
32172,32172,past,subjunctive,perfect,non-causative,affirmative,2p,passive,intrans,be-van-i-ast-*abion*,bevɛnijɛstɛˈbijon,ɛ,0.647059,1


In [328]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:


        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'First Character']] 
        y = all_verbs[feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['First Character'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['First Character'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [329]:
log_with_first_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [330]:
log_with_first_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9996484752614465,
  'confusion matrix': array([[7960,    2],
         [   2, 3415]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9912118815361631,
  'confusion matrix': array([[ 186,    0,   42],
         [   6, 7927,    0],
         [  50,    2, 3166]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9737235257931277,
  'confusion matrix': array([[3091,   10,    0,    0],
         [  11, 3365,  114,    0],
         [   0,  164, 3012,    0],
         [   0,    0,    0, 1612]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999121188153617,
  'confusion matrix': array([[3579,    0],
         [   1, 7799]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9991211881536163,
  'confusion matrix': array([[5715,    2],
         [   8, 5654]])},
 'Person': {'model': LogisticRegression(max_iter=10000

In [331]:
all_verbs['Last Character'] = all_verbs['Generated Verb-SR'].apply(lambda word: word[-1]) #before : goes input variable

In [332]:
le = LabelEncoder()

all_verbs['Last Character Number'] = le.fit_transform(all_verbs['Last Character'])

In [333]:
all_verbs['Last Character Number']

0        15
1         5
2         3
3        15
4        15
         ..
34475    15
34476    15
34477     8
34478     8
34479     8
Name: Last Character Number, Length: 34480, dtype: int64

In [334]:
le = LabelEncoder()
all_verbs_encoded = all_verbs
all_verbs_encoded = all_verbs_encoded.apply(lambda col: le.fit_transform(col))

In [335]:
all_verbs_encoded

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number
0,0,1,1,0,1,1,1,1,0,9662,67,1,40,0,15,15
1,1,1,1,0,1,1,3,1,0,9664,62,1,46,0,5,5
2,2,1,1,0,1,1,6,1,0,9661,66,1,43,0,3,3
3,3,1,1,0,1,1,5,1,0,9665,63,1,41,0,15,15
4,4,1,1,0,1,1,0,1,0,9666,69,1,40,0,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34475,34475,0,2,2,1,0,6,0,1,30631,30562,6,60,0,15,15
34476,34476,0,2,2,1,0,5,0,1,30631,30562,6,60,0,15,15
34477,34477,0,2,2,1,0,0,0,1,30629,30559,6,55,0,8,8
34478,34478,0,2,2,1,0,2,0,1,30630,30558,6,55,0,8,8


In [336]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:
        model = LogisticRegression(max_iter=100000) 

        X = all_verbs[['Generated Verb-SR', 'Last Character Number']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [337]:
logistic_reg_with_last_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [338]:
logistic_reg_with_last_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993848317075315,
  'confusion matrix': array([[7957,    5],
         [   2, 3415]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9912997627208014,
  'confusion matrix': array([[ 187,    0,   41],
         [   5, 7927,    1],
         [  50,    2, 3166]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.973811406977766,
  'confusion matrix': array([[3091,   10,    0,    0],
         [   9, 3366,  115,    0],
         [   0,  164, 3012,    0],
         [   0,    0,    0, 1612]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999121188153617,
  'confusion matrix': array([[3579,    0],
         [   1, 7799]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993848317075315,
  'confusion matrix': array([[5715,    2],
         [   5, 5657]])},
 'Person': {'model': LogisticRegression(max_iter=100000

In [339]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:
        model = LogisticRegression(max_iter=100000)
        X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [340]:
log_reg_with_first_and_last_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [341]:
log_reg_with_first_and_last_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993848317075315,
  'confusion matrix': array([[7957,    5],
         [   2, 3415]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9912118815361631,
  'confusion matrix': array([[ 184,    0,   44],
         [   5, 7927,    1],
         [  48,    2, 3168]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.973811406977766,
  'confusion matrix': array([[3091,   10,    0,    0],
         [   9, 3366,  115,    0],
         [   0,  164, 3012,    0],
         [   0,    0,    0, 1612]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999121188153617,
  'confusion matrix': array([[3579,    0],
         [   1, 7799]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994727128921698,
  'confusion matrix': array([[5715,    2],
         [   4, 5658]])},
 'Person': {'model': LogisticRegression(max_iter=100000

In [342]:
# as a last feature, I want to see if the number of vowels is relevant to the predictions

In [343]:
all_verbs['Vowels Quantity'] = all_verbs['Generated Verb-SR'].apply(lambda word: len(re.findall(r'[aouiøɛʌe]', word))) #before : goes input variable

In [344]:
all_verbs['Vowels Quantity']

0        6
1        5
2        5
3        6
4        6
        ..
34475    5
34476    5
34477    5
34478    5
34479    5
Name: Vowels Quantity, Length: 34480, dtype: int64

In [345]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34475,34475,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
34476,34476,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
34477,34477,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
34478,34478,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [346]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:

        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'Vowels Quantity']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Vowels Quantity'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train) 
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info
  


In [347]:
log_reg_with_vowel_quantity = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [348]:
log_reg_with_vowel_quantity

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9996484752614465,
  'confusion matrix': array([[7960,    2],
         [   2, 3415]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9912997627208014,
  'confusion matrix': array([[ 184,    1,   43],
         [   5, 7927,    1],
         [  48,    1, 3169]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9734598822392125,
  'confusion matrix': array([[3090,   11,    0,    0],
         [  12, 3363,  115,    0],
         [   0,  164, 3012,    0],
         [   0,    0,    0, 1612]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999121188153617,
  'confusion matrix': array([[3579,    0],
         [   1, 7799]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9991211881536163,
  'confusion matrix': array([[5713,    4],
         [   6, 5656]])},
 'Person': {'model': LogisticRegression(max_iter=10000

In [349]:
## putting all features together in a LR model:

In [350]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
    y = all_verbs[features] 
    X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X, y, test_size = 0.33, random_state = 1)
    count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
    X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
    X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
    for feature in features:
        model = LogisticRegression(max_iter=100000) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train_pre_encoded[feature]) 
        y_test = le.transform(y_test_pre_encoded[feature])
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info
    # this code is imroved: most reliable for my classification models (it is faster as some parts have gone out of the scope of the function)


In [351]:
log_reg_with_three_added_features = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [352]:
log_reg_with_three_added_features

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994727128921698,
  'confusion matrix': array([[7958,    4],
         [   2, 3415]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9911240003515247,
  'confusion matrix': array([[ 184,    0,   44],
         [   6, 7926,    1],
         [  48,    2, 3168]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9737235257931277,
  'confusion matrix': array([[3091,   10,    0,    0],
         [  10, 3365,  115,    0],
         [   0,  164, 3012,    0],
         [   0,    0,    0, 1612]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999121188153617,
  'confusion matrix': array([[3579,    0],
         [   1, 7799]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994727128921698,
  'confusion matrix': array([[5715,    2],
         [   4, 5658]])},
 'Person': {'model': LogisticRegression(max_iter=10000

In [353]:
all_verbs.head()

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6


In [354]:
tense_model = log_reg_with_three_added_features['Tense']['model'] #separating the models from each other

In [355]:
X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] #select the list by two square brackets
y = all_verbs['Tense'] 
words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) # splitting X twice , in one of them keep the original in one vectorized. 
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded) 
y_test = le.transform(y_test_pre_encoded)
print(le.classes_)

['past' 'present']


In [356]:
# I only pulled out tense, and now we are predict on the test set top see where the misclassifications occured
tense_model.predict(X_test) # compare these with y_test

array([0, 1, 1, ..., 0, 0, 0])

In [357]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [358]:
y_test !=  tense_model.predict(X_test) # pairwise comparison

array([False, False, False, ..., False, False, False])

In [359]:
words_test[y_test !=  tense_model.predict(X_test)] # these will keep only the rows that do not agree

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity
9372,mømaˈrdɛ,15,1,3
12044,merbetiˈ,19,1,3
12046,merbetiˈjɛ,15,1,4
9270,mømaˈndɛ,15,1,3
9386,bømaˈrdɛ,15,1,3
9284,bømaˈndɛ,15,1,3



aspect_model = log_reg_with_three_added_features['Aspect']['model']
X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) 
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded)
y_test = le.transform(y_test_pre_encoded)
print(le.classes_)
aspect_model.predict(X_test)
aspect_mismatches = words_test[y_test !=  aspect_model.predict(X_test)]
aspect_mismatches['y_train_pre_encoded'] = y_train_pre_encoded
aspect_mismatches.head()

In [360]:
type(words_test)

pandas.core.frame.DataFrame

In [361]:
type(X_test)

scipy.sparse.coo.coo_matrix

In [362]:
tense_model.predict(X_test)

array([0, 1, 1, ..., 0, 0, 0])

In [363]:
le.classes_

array(['past', 'present'], dtype=object)

words_test['predicted'] = modaliy_model.predict(X_test) #made a new column

words_test['predicted_aspect'] = words_test['predicted'].apply(lambda x: le.classes_[x]) #index lookup and doing the opposite of vectorization. Give me the class at index 0 or 1 ...

words_test['actual_aspect'] = all_verbs.iloc[words_test.index]['Aspect'] #word test preserves the indexes from the original data frame (see on the left): the indexe are not in a paarticular order. so you can use this index to go back to 

original df and pass that indexes to df and look up what the values are for the rows with that index, we only care about values of aspect columns, we wanna get the aspect value


In [364]:
def LG_model():
        dataframes = {}
        features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
        for feature in features:
                feature_model = log_reg_with_three_added_features[feature]['model']
                X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
                y = all_verbs[feature]
                words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) 
                count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
                X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
                    
                X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
                le = LabelEncoder()
                y_train = le.fit_transform(y_train_pre_encoded)
                y_test = le.transform(y_test_pre_encoded)
                print(le.classes_)
                words_test['predicted'] = feature_model.predict(X_test) 
                words_test['predicted ' + feature] = words_test['predicted'].apply(lambda x: le.classes_[x])
                words_test['actual ' + feature] = pd.Series(y_test).apply(lambda x: le.classes_[x]).values
                dataframes[feature] = words_test
        return dataframes


In [365]:
dfs = LG_model()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [366]:
dfs['Tense']

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense
18503,neˈved͡ʒijɛstiʃɛ,15,1,6,0,past,past
20059,menʌˈvid͡ʒijɛ,15,1,5,1,present,present
23767,omneˈkɛʃi,5,0,4,1,present,present
32619,ʌnviˈt͡ʃt͡ʃon,8,0,3,0,past,past
30024,ʌniˈpit͡ʃdenastemɛ,15,0,7,0,past,past
...,...,...,...,...,...,...,...
23099,mʌvid͡ʒijɛˈstimɛ,15,1,6,0,past,past
18981,neˈvʌzdinijʌ,17,1,5,1,present,present
24307,onpʌmaˈstemɛ,15,0,5,0,past,past
24413,oneˈpʌmijɛstiʃɛ,15,0,7,0,past,past


In [367]:
base_df = dfs['Tense'].copy()
base_df['predicted Modality'] = dfs['Modality']['predicted Modality']
base_df['actual Modality'] = dfs['Modality']['actual Modality']
base_df['predicted Aspect'] = dfs['Aspect']['predicted Aspect']
base_df['actual Aspect'] = dfs['Aspect']['actual Aspect']
base_df['predicted Causation'] = dfs['Causation']['predicted Causation']
base_df['actual Causation'] = dfs['Causation']['actual Causation']
base_df['predicted Polarity'] = dfs['Polarity']['predicted Polarity']
base_df['actual Polarity'] = dfs['Polarity']['actual Polarity']
base_df['predicted Person'] = dfs['Person']['predicted Person']
base_df['actual Person'] = dfs['Person']['actual Person']
base_df['predicted Transitivity'] = dfs['Transitivity']['predicted Transitivity']
base_df['actual Transitivity'] = dfs['Transitivity']['actual Transitivity']

In [368]:
base_df.shape

(11379, 19)

In [369]:
base_df

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense,predicted Modality,actual Modality,predicted Aspect,actual Aspect,predicted Causation,actual Causation,predicted Polarity,actual Polarity,predicted Person,actual Person,predicted Transitivity,actual Transitivity
18503,neˈved͡ʒijɛstiʃɛ,15,1,6,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,negative,negative,2s,2s,intrans,intrans
20059,menʌˈvid͡ʒijɛ,15,1,5,1,present,present,indicative,indicative,imperfective,imperfective,non-causative,non-causative,negative,negative,3sf,3sf,trans,trans
23767,omneˈkɛʃi,5,0,4,1,present,present,indicative,indicative,imperfective,imperfective,non-causative,non-causative,negative,negative,2s,2s,trans,trans
32619,ʌnviˈt͡ʃt͡ʃon,8,0,3,0,past,past,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,3p,3p,trans,trans
30024,ʌniˈpit͡ʃdenastemɛ,15,0,7,0,past,past,indicative,indicative,neutral,neutral,causative,causative,negative,negative,1s,1s,trans,trans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23099,mʌvid͡ʒijɛˈstimɛ,15,1,6,0,past,past,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,1s,1s,intrans,intrans
18981,neˈvʌzdinijʌ,17,1,5,1,present,present,subjunctive,subjunctive,neutral,neutral,causative,causative,negative,negative,2p,2p,intrans,intrans
24307,onpʌmaˈstemɛ,15,0,5,0,past,past,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,1s,1s,trans,trans
24413,oneˈpʌmijɛstiʃɛ,15,0,7,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,negative,negative,2s,2s,intrans,intrans


In [370]:
base_df.to_csv('./base_df.csv', encoding = 'utf-8-sig', index = False)

In [371]:
base_df

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense,predicted Modality,actual Modality,predicted Aspect,actual Aspect,predicted Causation,actual Causation,predicted Polarity,actual Polarity,predicted Person,actual Person,predicted Transitivity,actual Transitivity
18503,neˈved͡ʒijɛstiʃɛ,15,1,6,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,negative,negative,2s,2s,intrans,intrans
20059,menʌˈvid͡ʒijɛ,15,1,5,1,present,present,indicative,indicative,imperfective,imperfective,non-causative,non-causative,negative,negative,3sf,3sf,trans,trans
23767,omneˈkɛʃi,5,0,4,1,present,present,indicative,indicative,imperfective,imperfective,non-causative,non-causative,negative,negative,2s,2s,trans,trans
32619,ʌnviˈt͡ʃt͡ʃon,8,0,3,0,past,past,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,3p,3p,trans,trans
30024,ʌniˈpit͡ʃdenastemɛ,15,0,7,0,past,past,indicative,indicative,neutral,neutral,causative,causative,negative,negative,1s,1s,trans,trans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23099,mʌvid͡ʒijɛˈstimɛ,15,1,6,0,past,past,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,1s,1s,intrans,intrans
18981,neˈvʌzdinijʌ,17,1,5,1,present,present,subjunctive,subjunctive,neutral,neutral,causative,causative,negative,negative,2p,2p,intrans,intrans
24307,onpʌmaˈstemɛ,15,0,5,0,past,past,indicative,indicative,imperfective,imperfective,non-causative,non-causative,affirmative,affirmative,1s,1s,trans,trans
24413,oneˈpʌmijɛstiʃɛ,15,0,7,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,negative,negative,2s,2s,intrans,intrans


In [372]:
aspect_df = dfs['Aspect']

In [373]:
aspect_df[aspect_df['Generated Verb-SR'] == 'ʌriˈtemɛ']

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Aspect,actual Aspect


In [374]:
all_verbs[all_verbs['Generated Verb-SR'] == 'ʌriˈtemɛ']

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
30631,30631,past,indicative,neutral,non-causative,affirmative,1s,active,trans,ʌ-*rit*-em(ɛ),ʌriˈtemɛ,i,0.375,0,ɛ,15,4
